In [2]:
import mne
import numpy as np
import os

# 设置文件路径
data_folder = "D:/P/EEG_Process_Project/NewDataSet/"
file_names = ["Female1-post.set", "Female2-post.set", "Female3-post.set", "Data1-post.set", "Data2-post.set", "Male1-post.set", "Female7-post.set", "Female8-post.set", "Male2-post.set", "Male4-post.set", "Male5-post.set", "Male6-post.set"]
dataLabel = [0,0,0,1,0,1,0,0,1,1,1,1]
# 用于存储每段 EEG 数据的列表
eeg_data_list = []

# 读取每个文件中的 EEG 数据
for file_name in file_names:
    file_path = os.path.join(data_folder, file_name)
    raw = mne.io.read_raw_eeglab(file_path, preload=True)
    
    # 提取前 32 个通道是 EEG 通道
    eeg_data = raw.get_data()[:32, :]
    eeg_data_list.append(eeg_data)

# 将列表中的 EEG 数据合并为一个 numpy 数组
data = np.stack(eeg_data_list, axis=0)

Reading D:\P\EEG_Process_Project\NewDataSet\Female1-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Female2-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Female3-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Data1-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Data2-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Male1-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Female7-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Female8-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...


C:\Users\Keren\AppData\Local\Temp\ipykernel_20464\2942066813.py:15: RuntimeWarning: Data file name in EEG.data (TestData2-post.fdt) is incorrect, the file name must have changed on disk, using the correct file name (Data2-post.fdt).
  raw = mne.io.read_raw_eeglab(file_path, preload=True)


Reading D:\P\EEG_Process_Project\NewDataSet\Male2-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Male4-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Male5-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...
Reading D:\P\EEG_Process_Project\NewDataSet\Male6-post.fdt
Reading 0 ... 34499  =      0.000 ...    68.998 secs...


In [3]:
import pywt
import matplotlib.pyplot as plt
import numpy as np
import os

TimeStart = 0                     # start time
TimeEnd = 69                       # end time 
sampleRate = 500
samplePointStart=TimeStart*sampleRate
samplePointEnd=TimeEnd*sampleRate
wavename = 'morl'    
totalscal = 69    # scale 
fc = pywt.central_frequency(wavename) #  central frequency
cparam = 2 * fc * totalscal
scales = cparam/np.arange(1,totalscal+1)
t = np.arange(TimeStart,TimeEnd,1.0/sampleRate)

for i in range(data.shape[0]):  # 每一条数据
    cwt_data = data[i, :, :]  

    channels = 32
    # 初始化小波时频图
    cwtmatr = np.zeros((channels, len(scales), len(t)))
    
    # 对每个通道进行小波变换
    for j in range(channels):
        cwtmatr[j, :, :], frequencies = pywt.cwt(cwt_data[j, :], scales, wavename, 1.0/sampleRate)
    
    # 将所有通道的小波时频图合并

    avg_cwtmatr = np.mean(cwtmatr, axis=0)
    # print(cwtmatr.shape)
    # print(avg_cwtmatr.shape)
    #创建存储细分图像的子文件夹
    if dataLabel[i] == 0:
        folderpath = r'New_Female/' + str(i)
    elif dataLabel[i] == 1:
        folderpath = r'New_Male/' + str(i)
    os.makedirs(folderpath, exist_ok=True)

    for k in range(6):  # 细分图像
        fig = plt.figure()

        start_col = k * (avg_cwtmatr.shape[1] // 6)
        end_col = (k + 1) * (avg_cwtmatr.shape[1] // 6)

        plt.contourf(avg_cwtmatr[:, start_col:end_col])
        plt.xticks([])  # remove x
        plt.yticks([])  # remove y
        plt.axis('off') # remove axis
        fig.set_size_inches(800/100.0, 600/100.0)  # set pixels width*height
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
        plt.gca().yaxis.set_major_locator(plt.NullLocator())
        plt.subplots_adjust(top=1, bottom=0, left=0, right=1, hspace=0, wspace=0) 
        plt.margins(0, 0)

        filepath = folderpath + '/' + str(i) + '_' + str(k) + '_sub.jpg'
        fig.savefig(filepath)
        fig.clear()

print('wavelet transform completed')


C:\Users\Keren\AppData\Local\Temp\ipykernel_20464\150784676.py:42: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure()


wavelet transform completed


<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

...

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [4]:
from PIL import ImageEnhance
import os
import numpy as np
from PIL import Image

def brightnessEnhancement(root_path,img_name): #亮度增强
    image = Image.open(os.path.join(root_path, img_name))
    enh_bri = ImageEnhance.Brightness(image)
    # brightness = 1.1+0.4*np.random.random()#取值范围1.1-1.5
    brightness = 1.5
    image_brightened = enh_bri.enhance(brightness)
    return image_brightened


def contrastEnhancement(root_path, img_name):  # 对比度增强
    image = Image.open(os.path.join(root_path, img_name))
    enh_con = ImageEnhance.Contrast(image)
    # contrast = 1.1+0.4*np.random.random()#取值范围1.1-1.5
    contrast = 1.5
    image_contrasted = enh_con.enhance(contrast)
    return image_contrasted

def rotation(root_path, img_name):
    img = Image.open(os.path.join(root_path, img_name))
    random_angle = np.random.randint(-2, 2)*90
    if random_angle==0:
        rotation_img = img.rotate(-90) #旋转角度
    else:
        rotation_img = img.rotate( random_angle)  # 旋转角度
    # rotation_img.save(os.path.join(root_path,img_name.split('.')[0] + '_rotation.jpg'))
    return rotation_img

def flip(root_path,img_name):   #翻转图像
    img = Image.open(os.path.join(root_path, img_name))
    filp_img = img.transpose(Image.FLIP_LEFT_RIGHT)
    # filp_img.save(os.path.join(root_path,img_name.split('.')[0] + '_flip.jpg'))
    return filp_img

def createImage(imageDir,saveDir):
    i=0
    for name in os.listdir(imageDir):
        i=i+1
        saveName="cesun"+str(i)+".jpg"
        saveImage=contrastEnhancement(imageDir,name)
        saveImage.save(os.path.join(saveDir,saveName))
        saveName1 = "flip" + str(i) + ".jpg"
        saveImage1 = flip(imageDir,name)
        saveImage1.save(os.path.join(saveDir, saveName1))
#         saveName2 = "brightnessE" + str(i) + ".jpg"
#         saveImage2 = brightnessEnhancement(imageDir, name)
#         saveImage2.save(os.path.join(saveDir, saveName2))
        saveName3 = "rotate" + str(i) + ".jpg"
        saveImage = rotation(imageDir, name)
        saveImage.save(os.path.join(saveDir, saveName3))


In [5]:
# label 1
imageDir="New_Female/" #要改变的图片的路径文件夹
saveDir="New_Female_aug/"   #数据增强生成图片的路径文件夹
createImage(imageDir,saveDir)

# label 2
imageDir="New_Male/" #要改变的图片的路径文件夹
saveDir="New_Male_aug/"   #数据增强生成图片的路径文件夹
createImage(imageDir,saveDir)
